In [1]:
import numpy as np
import itertools
import pygame
import random
import copy

from pygame.locals import (
    K_UP,
    K_DOWN,
    K_LEFT,
    K_RIGHT,
    K_ESCAPE,
    KEYDOWN,
    QUIT,
)

pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
CARS = {
    "types": ["car", "bus", "taxi", "lorry"],
    "probability": [0.8, 0.1, 0.05, 0.05]
}

class CarType:
    def __init__(self, type=None):
        if type is None:
            self.type = random.choices(CARS["types"], weights=CARS["probability"], k=1)[0]
        else:
            self.type = type

    def get_length(self):
        if self.type == "car" or self.type == "taxi":
            return 50 # 5 meters
        if self.type == "bus":
            return 120 # 12 meters
        if self.type == "lorry":
            return 160   # 15 meters
        return 0
    
    def get_color(self):
        if self.type == "car":
            return pygame.Color("blue")
        if self.type == "taxi":
            return pygame.Color("yellow")
        if self.type == "bus":
            return pygame.Color("green")
        if self.type == "lorry":
            return pygame.Color("red")
        
    def get_acceleration(self):
        if self.type == "car" or self.type == "taxi":
            return 20 # 2 m/s^2
        if self.type == "bus":
            return 15 # 1.5 m/s^2
        if self.type == "lorry":
            return 12   # 1.2 m/s^2
        return 0
    
    def get_max_speed(self):
        if self.type == "car" or self.type == "taxi":
            return 250 # ~ 90kmh
        if self.type == "bus":
            return 220 # ~ 80kmh
        if self.type == "lorry":
            return 190   # ~ 70kmh
        return 0

In [3]:
class Car:
    def __init__(self, min_speed_on_lane, max_speed_on_lane, position):
        self.car_type = CarType()
        self.min_speed = 0
        self.max_speed = self.car_type.get_max_speed()
        self.speed = random.randint(min_speed_on_lane, min(max_speed_on_lane, self.max_speed))
        self.position = position
        self.car_size = self.car_type.get_length()
    
    def increase_speed(self):
        if self.speed < self.max_speed:
            self.speed += self.car_type.get_acceleration()
    
    def set_speed(self, new_speed):
        self.speed = min(new_speed, self.max_speed)
        
    def randomize_speed(self, probability):
        if self.speed > 1 and random.random() < probability:
            self.speed -= random.choice(range(10,25))

    def move(self):
        self.position += (self.speed // 10)

In [4]:
class CROSSROAD:
    class LANE:
        def __init__(self, lenght, start_point, new_car_probabulity, min_speed=None, max_speed=None):
            self.lenght = lenght
            self.start_point = start_point
            self.new_car_probabulity = new_car_probabulity
            self.min_speed = min_speed
            self.max_speed = max_speed

    lane1 = LANE(lenght=4000, start_point=0, new_car_probabulity=.07, min_speed=190, max_speed=250)  # m/s into km/h ~ 70km/h - 90km/h
    lane2 = LANE(lenght=4000, start_point=0, new_car_probabulity=.1, min_speed=190, max_speed=250)
    lane3 = LANE(lenght=4000, start_point=0, new_car_probabulity=.07, min_speed=110, max_speed=170)  # ~ 40 km/h - 60km/h 
    lane4 = LANE(lenght=2500, start_point=0, new_car_probabulity=.05, min_speed=110, max_speed=140)  # ~ 40km/h - 50km/h
    left_turn_line = LANE(lenght=1500, start_point=2500, new_car_probabulity=0)

    class LIGHTS:
        class LIGHT_CYCLE:
            def __init__(self, time, straight, turn_left):
                self.time = time
                self.straight = straight
                self.turn_left = turn_left
        cycle = [
            LIGHT_CYCLE(time=450, straight=True, turn_left=False),
            LIGHT_CYCLE(time=150, straight=True, turn_left=True),
            LIGHT_CYCLE(time=450, straight=False, turn_left=False),
        ]
        last_time_change = 0

        def curr_cycle(self):
            return self.cycle[0]

class Model:
    def __init__(self):
        self.time = 0
        self.road = [CROSSROAD.left_turn_line, CROSSROAD.lane1, CROSSROAD.lane2, CROSSROAD.lane3, CROSSROAD.lane4]
        self.cars = [[],[],[],[],[]] # cars on each line 
        # self.lanes_capacity = [[False] * lane.lenght for lane in self.road] #where are taken places on the road
        self.lights = CROSSROAD.LIGHTS()
        
        self.car_img = pygame.image.load('car-icon.png')
        self.bus_img = pygame.image.load('bus-icon.png')
        self.taxi_img = pygame.image.load('taxi-icon.png')
        self.lorry_img = pygame.image.load('lorry-icon.png')
    
    def randomlyAddNewCar(self):
        for i, line in enumerate(self.road):
            if random.random() < line.new_car_probabulity and (not self.cars[i] or self.cars[i][-1].position != 0):
                self.cars[i].append(Car(line.min_speed, line.max_speed, position=0))
            
    
    def update(self):
        self.time += 1

        # Change Lights color
        if (self.time - self.lights.last_time_change) % self.lights.curr_cycle().time == 0:
            last_cycle = self.lights.cycle.pop(0)
            self.lights.cycle.append(last_cycle)

        # Acceleration
        for car_line in self.cars:
            for car in car_line:
                car.increase_speed()

        # First car slow down on red
        for line_number, lane_with_cars in enumerate(self.cars):
            if not lane_with_cars: continue
            car = lane_with_cars[0] # first car
            distance_to_lights = self.road[line_number].lenght - car.position - 40 # 4 meters 
            if distance_to_lights < car.speed:
                if line_number == 0 and not self.lights.curr_cycle().turn_left:
                    car.set_speed(distance_to_lights)
                elif not self.lights.curr_cycle().straight:
                    car.set_speed(distance_to_lights)

        # Slowing Down
        for lane_number, lane_with_cars in enumerate(self.cars):
            for i in range(1, len(lane_with_cars)): # enumerate cars but without the first one on the lane 
                car = lane_with_cars[i]
                next_car = lane_with_cars[i-1]
                distance = (next_car.position - next_car.car_size) - car.position - 5 # 50cm safe distance beetween 
                if distance < car.speed:
                    car.set_speed(distance)
                    # if car.speed < (car.max_speed // 2):
                    #     if not any(self.lanes_capacity[lane_number - 1][max(car.position-car.car_size-40, 0) : min(car.position + distance + 40, 4000)]):

                
        # Randomization
        for car_line in self.cars:
            for car in car_line:
                car.randomize_speed(probability=0.1)

        # Car motion
        for line_index, car_line in enumerate(self.cars):
            # self.lanes_capacity[line_index] = [False] * self.road[line_index].lenght
            for car_index, car in enumerate(car_line):
                car.move()
                #self.lanes_capacity[line_index][max(0, car.position - car.car_size) : car.position] = [True] * len(range(max(0, car.position - car.car_size), car.position))
                if car.position >= self.road[line_index].lenght:
                    car_line.pop(car_index)

        self.randomlyAddNewCar()
        
        
    def draw(self, screen, w_width, w_height):
        black = (0,0,0)
        white = (255,255,255)  
        grey = (128,128,128)
        block_width = w_width / max(self.road, key=lambda lane: lane.lenght).lenght
        block_height = 30

        # draw road
        pos_x = 0
        pos_y = (w_height-(5*block_height))//2
        
        for line_number, line in enumerate(self.road):
            # LANES
            rect = pygame.Rect(
                pos_x + (block_width * line.start_point),
                pos_y + (block_height * line_number),
                block_width * line.lenght,
                block_height
            )
            pygame.draw.rect(screen, grey, rect, 0)

    
        for line_num, car_line in enumerate(self.cars):
            for car in car_line:
                # CARS
                x = (block_width) * (car.position - car.car_size)
                y = pos_y + (block_height * line_num)
                rect = pygame.Rect(x, y, int(block_width * car.car_size), block_height)
                
                pygame.draw.rect(screen, car.car_type.get_color(), rect, 0)
                if car.car_type.type == "bus":
                    screen.blit(pygame.transform.scale(self.bus_img, (int(block_width * car.car_size), block_height)), rect)
                elif car.car_type.type == "taxi":
                    screen.blit(pygame.transform.scale(self.taxi_img, (int(block_width * car.car_size), block_height)), rect)
                elif car.car_type.type == "lorry":
                    screen.blit(pygame.transform.scale(self.lorry_img, (int(block_width * car.car_size), block_height)), rect)
                else:
                    screen.blit(pygame.transform.scale(self.car_img, (int(block_width * car.car_size), block_height)), rect)


        for line_number, line in enumerate(self.road):
            # LIGHTS 
            x = w_width - 20
            y = pos_y + (block_height * line_number)
            rect = pygame.Rect(x, y, 20, block_height)
            if (self.lights.curr_cycle().turn_left and line_number == 0) or (self.lights.curr_cycle().straight and line_number != 0):
                pygame.draw.rect(screen, pygame.Color("green"), rect, 0)
            else:
                pygame.draw.rect(screen, pygame.Color("red"), rect, 0)

        pygame.display.flip()

In [5]:
def run():
    import time
    pygame.init()
    w_width = 1800
    w_height = 300
    screen = pygame.display.set_mode([w_width, w_height])
    model = Model()
    screen.fill((255, 255, 255))

        
    running = True
    stop = False
    model.draw(screen, w_width, w_height)
    while running:
        for event in pygame.event.get():   
            if event.type == QUIT:
                running = False
            
            if event.type == KEYDOWN:
                if event.key == K_RIGHT:
                    stop = not stop
                    # for carl in model.cars:
                    #     for car in carl:
                    #         print(f"({car.position}, {car.speed}), ", end='')
                    #     print()
        if not stop:
            model.update()  # Update the model
            model.draw(screen, w_width, w_height)  # Redraw the screen
            pygame.display.flip()  # Update the display
        time.sleep(0.1)  # Wait for 0.1 second

    pygame.quit()

In [6]:
run()